# Glucose Segment Plot

This notebook retrieves a "segment" of data, defined as the points between two timestamps, and plots a simple line chart of the data with the target range on the chart highlighted.

Before attempting to run the notebook:

- Make sure the variables defined in "config.ipynb" are set correctly
- Set the reporting date range and export options in the first code cell

In [ ]:
from datetime import datetime

# Reporting date range
start = datetime(2025, 1, 1, 0, 0, 0)
end = datetime(2025, 1, 1, 0, 0, 0)

# Report title
report_title = ""

# Whether to export the data to a spreadsheet
export_spreadsheet = True

# Export format for the chart:
# PNG     - export as PNG image
# PDF     - export as PDF file
# <blank> - do not export
chart_export_format = "PNG"

In [ ]:
%run ../api.ipynb
%run ../config.ipynb
%run ../export.ipynb

In [ ]:
# Log in to the service, get the person ID and retrieve the data
token = authenticate(url, username, password)
person_id = get_person_id(url, token, firstnames, surname)
df = get_blood_glucose_measurements(url, token, person_id, start, end)

# Preview the data
df.head()

In [ ]:
# Calculate minutes from the first point
df["minutes"] = (df["date"] - df["date"].iloc[0]).dt.total_seconds() / 60

# Preview the data
df.head()

In [ ]:
from pathvalidate import sanitize_filepath

# Prepend the start timestamp onto the report to give the basis of the export file
timestamp = start.strftime("%Y-%m-%d %H-%M")
export_file_name = f"{timestamp} {report_title}"

# Now sanitise it
export_file_name = sanitize_filepath(export_file_name)

# Export the data to a spreadsheet
if export_spreadsheet:
    export_to_spreadsheet(export_file_name, { export_file_name: df })

In [ ]:
import matplotlib.pyplot as plt

# Plot setup
plt.figure(figsize=(12, 6))

# Plot daily mean glucose
plt.plot(df["minutes"], df["level"], label="Glucose (mmol/L)")

# Add target range band: 3.9–10.0 mmol/L
plt.axhspan(3.9, 10.0, color="green", alpha=0.1, label="Target Range (3.9–10.0 mmol/L)")

# Construct the plot title
timestamp = start.strftime("%Y-%m-%d %H:%M")
title = f"{report_title} - {timestamp}"

# Formatting
plt.title(title)
plt.xlabel("Minutes")
plt.ylabel("Glucose (mmol/L)")
plt.xticks(rotation=45)
plt.grid(True)

# Legend below chart
plt.legend(loc="upper center", bbox_to_anchor=(0.5, -0.25), ncol=2)
plt.tight_layout()

# Export to PNG or PDF, if required
export_chart(export_file_name, None, chart_export_format)

# Show the plot
plt.show()
